In [1]:

# coding: utf-8

# In[1]:


import cv2
cv2.__version__


# In[11]:


import numpy as np
cv2.startWindowThread()
cv2.namedWindow('showboxes')
img = cv2.imread("frame1.png")


# In[3]:


# test plotting
def imshow(img):  
    cv2.imshow('showboxes', cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    cv2.waitKey()
imshow(img)



In [43]:
def showboxes(imagename):
    img = image = cv2.imread(imagename)
    # step 1, extract the right kind of blue
    image = img
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    threshold  = np.array([[100,0,0],[140,255,255]])
    mask = cv2.inRange(hsv, threshold[0,:], threshold[1,:])
    masked = cv2.bitwise_and(hsv, hsv, mask=mask)
    blue = cv2.cvtColor(masked, cv2.COLOR_HSV2BGR)
    del  masked
    bluewhite = cv2.bitwise_or(blue, cv2.bitwise_not(np.stack(3* [mask], axis=2)))
    #imshow(bluewhite)


    # In[59]:


    dst = np.zeros(image.shape, np.uint8)
    dst[:]=(240,240,240)


    # In[60]:


    # extract raw contours
    img = cv2.cvtColor(bluewhite, cv2.COLOR_BGR2GRAY)
    img = cv2.GaussianBlur(img, (3,3), 0)
    img = cv2.threshold(img, 60, 255, cv2.THRESH_BINARY)[1]
    img = cv2.bitwise_not(img)  # contours expects white image on black bg

    cnts = cv2.findContours(img.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[1]
    print(len(cnts))
    #cv2.drawContours(dst, cnts, -1, (0,255,0), 3)
    #plt.imshow(dst)


    # In[62]:


    # simplify contours, find one that is a rectangle
    dst[:] = (255,240,240)
    for c in cnts:
        peri = 0.01 * cv2.arcLength(c, True)  # approximate such that new perimeter is 1% of old one
        approx = cv2.approxPolyDP(c, peri, True)
        if len(approx) == 4:
                approx = np.reshape(approx,[4,2]) # drop 2nd dimension 
                xsize = approx[:,0].max() - approx[:,0].min()
                ysize = approx[:,1].max() - approx[:,1].min()
                if xsize > 300 and ysize > 150:
                 
                    cv2.drawContours(dst, [approx], -1, (0,255,0), 2)
                    break
    else:
        raise ValueError("No outer rectangle found")

    #imshow(dst)


    # In[8]:


    # apply perspective transform
    # http://www.pyimagesearch.com/2014/05/05/building-pokedex-python-opencv-perspective-warping-step-5-6/
    #approx
    #dst = np.array([[0,0],[799,0],[799,599],[0,599]], dtype = "float32")
    #M = cv2.getPerspectiveTransform(approx, dst)
    #warp = cv2.warpPerspective(img, M, (800, 600))


    # In[25]:


    # mask area outside outer rectangle on b/w image for contour finding
    mask = np.zeros(image.shape[:2], np.uint8)
    cv2.drawContours(mask, [approx], -1, 255, cv2.FILLED)
    img2 = cv2.bitwise_and(img, mask)
    #plt.imshow(img2)


    # In[13]:


    # mask blue white imge
    ##mask = np.zeros(image.shape[:2], np.uint8)
    #cv2.drawContours(mask, [approx], -1, (255), cv2.FILLED)
    #bluewhite2 = cv2.bitwise_or(bluewhite, cv2.bitwise_not(np.stack(3* [mask], axis=2)))
    #imshow(bluewhite2)



    # In[134]:


    # find inner rectangles
    _, cnts, hier = cv2.findContours(img2.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    contours = []
    for i in range(len(cnts)):
        contour = cnts[i]
        _,_,w,h = cv2.boundingRect(contour)
        if w>100 and h>50:
            contours.append(contour)
        else:
            hier[0, i, :] = -2 # mark for deletion    
    hier = np.delete(hier, np.where(hier[0,:,-1] == -2)[0], 1)
    
    assert len(contours) == 5, "there are five contours"
    assert hier[0,0,-1] == -1 , "first contour is the outer one, ie it has no parent"
    assert np.all(hier[0,1:5,-1] == 0) , " all other contours have the first contour as their parent"
    hier = np.delete(hier, 0, 1) # remove parent from list of hier
    contours = contours[1:] # remove parent from contours
    contours.sort(key=lambda a: a[:,0,1].min()) # sort contours by minimum y value (top down)


    # In[131]:


    # determine size of largest (topmost) bounding box
    _,_,bbw,bbh = cv2.boundingRect(contours[0])


    # In[135]:


    # for each inner box, add white mask and crop to a standard size box
    boxes = []
    kernel =  np.ones((5,5), np.uint8)
    counter = 0
    for contour in contours:
        _,_,w,h = cv2.boundingRect(contour)
        if w>100 and h>50:
            counter += 1
            # compute mask from contour by filling
            mask = np.zeros(image.shape[:2], np.uint8)
            cv2.drawContours(mask, [contour], -1, 255, cv2.FILLED)
            # make mask about 5px smaller on all sides
            mask = cv2.erode(mask, kernel)
            # create white box and copy image in where mask is true
            box = np.zeros_like(image)
            box.fill(255)
            box[mask == 255] = image[mask == 255] 
            # crop to bounding box
            bbx, bby, w, h = cv2.boundingRect(contour)
            assert bbw >= w and bbh >= h ,"boxes fit"
            box = box[bby:(bby+bbh), bbx:(bbx+bbw), :]
            boxes.append(box)
    assert counter == 4, "there are four inner boxes"


    # In[146]:


    #imshow(boxes[3])
    #imshow(np.concatenate(boxes))


    # In[ ]:






In [44]:
import glob, sys
for fn in glob.glob("stills-thu/frame*png"): 
    print(fn)
    try:
        showboxes(fn)
    except Exception as e:
        print("EXCEPTION:", e)
     
    

stills-thu/frame10.png
43
EXCEPTION: No outer rectangle found
stills-thu/frame100.png
23
EXCEPTION: No outer rectangle found
stills-thu/frame1000.png
72
stills-thu/frame1001.png
71
stills-thu/frame1002.png
74
stills-thu/frame1003.png
73
stills-thu/frame1004.png
79
stills-thu/frame1005.png
68
stills-thu/frame1006.png
79
stills-thu/frame1007.png
89
stills-thu/frame1008.png
73
stills-thu/frame1009.png
61
stills-thu/frame101.png
23
EXCEPTION: No outer rectangle found
stills-thu/frame1010.png
73
stills-thu/frame1011.png
78
stills-thu/frame1012.png
68
stills-thu/frame1013.png
83
stills-thu/frame1014.png
64
stills-thu/frame1015.png
69
stills-thu/frame1016.png
84
stills-thu/frame1017.png
76
stills-thu/frame1018.png
70
stills-thu/frame1019.png
73
stills-thu/frame102.png
24
EXCEPTION: No outer rectangle found
stills-thu/frame1020.png
73
stills-thu/frame1021.png
77
stills-thu/frame1022.png
72
stills-thu/frame1023.png
72
stills-thu/frame1024.png
70
stills-thu/frame1025.png
62
stills-thu/frame1026.

stills-thu/frame1236.png
74
stills-thu/frame1237.png
55
stills-thu/frame1238.png
69
stills-thu/frame1239.png
59
stills-thu/frame124.png
24
EXCEPTION: No outer rectangle found
stills-thu/frame1240.png
64
stills-thu/frame1241.png
65
stills-thu/frame1242.png
82
stills-thu/frame1243.png
66
stills-thu/frame1244.png
73
stills-thu/frame1245.png
61
stills-thu/frame1246.png
94
stills-thu/frame1247.png
65
stills-thu/frame1248.png
51
stills-thu/frame1249.png
67
stills-thu/frame125.png
23
EXCEPTION: No outer rectangle found
stills-thu/frame1250.png
74
stills-thu/frame1251.png
67
stills-thu/frame1252.png
67
stills-thu/frame1253.png
51
stills-thu/frame1254.png
73
stills-thu/frame1255.png
55
stills-thu/frame1256.png
52
stills-thu/frame1257.png
42
stills-thu/frame1258.png
46
stills-thu/frame1259.png
53
stills-thu/frame126.png
19
EXCEPTION: No outer rectangle found
stills-thu/frame1260.png
48
stills-thu/frame1261.png
53
stills-thu/frame1262.png
44
stills-thu/frame1263.png
46
stills-thu/frame1264.png
48

71
stills-thu/frame1472.png
78
stills-thu/frame1473.png
74
stills-thu/frame1474.png
58
stills-thu/frame1475.png
77
stills-thu/frame1476.png
70
stills-thu/frame1477.png
63
stills-thu/frame1478.png
58
stills-thu/frame1479.png
71
stills-thu/frame148.png
21
EXCEPTION: No outer rectangle found
stills-thu/frame1480.png
66
stills-thu/frame1481.png
62
stills-thu/frame1482.png
66
stills-thu/frame1483.png
63
stills-thu/frame1484.png
66
stills-thu/frame1485.png
56
stills-thu/frame1486.png
45
stills-thu/frame1487.png
58
stills-thu/frame1488.png
51
stills-thu/frame1489.png
55
stills-thu/frame149.png
20
EXCEPTION: No outer rectangle found
stills-thu/frame1490.png
53
stills-thu/frame1491.png
52
stills-thu/frame1492.png
53
stills-thu/frame1493.png
63
stills-thu/frame1494.png
55
stills-thu/frame1495.png
50
stills-thu/frame1496.png
47
stills-thu/frame1497.png
56
stills-thu/frame1498.png
64
stills-thu/frame1499.png
56
stills-thu/frame15.png
23
stills-thu/frame150.png
18
EXCEPTION: No outer rectangle foun

EXCEPTION: No outer rectangle found
stills-thu/frame1704.png
64
EXCEPTION: No outer rectangle found
stills-thu/frame1705.png
70
EXCEPTION: No outer rectangle found
stills-thu/frame1706.png
83
EXCEPTION: No outer rectangle found
stills-thu/frame1707.png
68
EXCEPTION: No outer rectangle found
stills-thu/frame1708.png
66
EXCEPTION: No outer rectangle found
stills-thu/frame1709.png
71
EXCEPTION: No outer rectangle found
stills-thu/frame171.png
45
EXCEPTION: No outer rectangle found
stills-thu/frame1710.png
77
EXCEPTION: No outer rectangle found
stills-thu/frame1711.png
83
EXCEPTION: No outer rectangle found
stills-thu/frame1712.png
71
EXCEPTION: No outer rectangle found
stills-thu/frame1713.png
77
EXCEPTION: No outer rectangle found
stills-thu/frame1714.png
76
EXCEPTION: No outer rectangle found
stills-thu/frame1715.png
73
EXCEPTION: No outer rectangle found
stills-thu/frame1716.png
73
EXCEPTION: No outer rectangle found
stills-thu/frame1717.png
82
EXCEPTION: No outer rectangle found
still

65
EXCEPTION: No outer rectangle found
stills-thu/frame1827.png
75
EXCEPTION: No outer rectangle found
stills-thu/frame1828.png
61
EXCEPTION: No outer rectangle found
stills-thu/frame1829.png
68
EXCEPTION: No outer rectangle found
stills-thu/frame183.png
41
EXCEPTION: No outer rectangle found
stills-thu/frame1830.png
71
EXCEPTION: No outer rectangle found
stills-thu/frame1831.png
61
EXCEPTION: No outer rectangle found
stills-thu/frame1832.png
55
EXCEPTION: No outer rectangle found
stills-thu/frame1833.png
62
EXCEPTION: No outer rectangle found
stills-thu/frame1834.png
67
EXCEPTION: No outer rectangle found
stills-thu/frame1835.png
64
EXCEPTION: No outer rectangle found
stills-thu/frame1836.png
56
EXCEPTION: No outer rectangle found
stills-thu/frame1837.png
52
EXCEPTION: No outer rectangle found
stills-thu/frame1838.png
45
EXCEPTION: No outer rectangle found
stills-thu/frame1839.png
61
EXCEPTION: No outer rectangle found
stills-thu/frame184.png
41
EXCEPTION: No outer rectangle found
sti

EXCEPTION: No outer rectangle found
stills-thu/frame1951.png
24
EXCEPTION: No outer rectangle found
stills-thu/frame1952.png
38
EXCEPTION: No outer rectangle found
stills-thu/frame1953.png
36
EXCEPTION: No outer rectangle found
stills-thu/frame1954.png
40
EXCEPTION: No outer rectangle found
stills-thu/frame1955.png
35
EXCEPTION: No outer rectangle found
stills-thu/frame1956.png
34
EXCEPTION: No outer rectangle found
stills-thu/frame1957.png
34
EXCEPTION: No outer rectangle found
stills-thu/frame1958.png
40
EXCEPTION: No outer rectangle found
stills-thu/frame1959.png
34
EXCEPTION: No outer rectangle found
stills-thu/frame196.png
41
EXCEPTION: No outer rectangle found
stills-thu/frame1960.png
40
EXCEPTION: No outer rectangle found
stills-thu/frame1961.png
32
EXCEPTION:  all other contours have the first contour as their parent
stills-thu/frame1962.png
37
EXCEPTION: No outer rectangle found
stills-thu/frame1963.png
44
EXCEPTION: No outer rectangle found
stills-thu/frame1964.png
50
EXCEPTI

stills-thu/frame308.png
60
stills-thu/frame309.png
57
stills-thu/frame31.png
31
EXCEPTION: No outer rectangle found
stills-thu/frame310.png
54
stills-thu/frame311.png
69
stills-thu/frame312.png
52
stills-thu/frame313.png
54
stills-thu/frame314.png
57
stills-thu/frame315.png
55
stills-thu/frame316.png
56
stills-thu/frame317.png
62
stills-thu/frame318.png
62
stills-thu/frame319.png
57
stills-thu/frame32.png
30
EXCEPTION: No outer rectangle found
stills-thu/frame320.png
60
stills-thu/frame321.png
70
stills-thu/frame322.png
61
stills-thu/frame323.png
60
stills-thu/frame324.png
65
stills-thu/frame325.png
68
stills-thu/frame326.png
62
stills-thu/frame327.png
53
stills-thu/frame328.png
53
stills-thu/frame329.png
72
stills-thu/frame33.png
38
EXCEPTION: No outer rectangle found
stills-thu/frame330.png
76
stills-thu/frame331.png
70
stills-thu/frame332.png
80
stills-thu/frame333.png
71
stills-thu/frame334.png
69
stills-thu/frame335.png
88
stills-thu/frame336.png
83
stills-thu/frame337.png
96
stil

157
stills-thu/frame546.png
151
stills-thu/frame547.png
178
stills-thu/frame548.png
178
stills-thu/frame549.png
171
stills-thu/frame55.png
41
EXCEPTION: No outer rectangle found
stills-thu/frame550.png
182
stills-thu/frame551.png
148
stills-thu/frame552.png
146
stills-thu/frame553.png
142
stills-thu/frame554.png
149
stills-thu/frame555.png
156
stills-thu/frame556.png
243
stills-thu/frame557.png
316
stills-thu/frame558.png
138
stills-thu/frame559.png
159
stills-thu/frame56.png
28
EXCEPTION: No outer rectangle found
stills-thu/frame560.png
190
stills-thu/frame561.png
165
stills-thu/frame562.png
181
stills-thu/frame563.png
196
stills-thu/frame564.png
169
stills-thu/frame565.png
176
stills-thu/frame566.png
198
EXCEPTION:  all other contours have the first contour as their parent
stills-thu/frame567.png
181
stills-thu/frame568.png
184
stills-thu/frame569.png
177
stills-thu/frame57.png
29
EXCEPTION: No outer rectangle found
stills-thu/frame570.png
174
stills-thu/frame571.png
159
stills-thu/f

39
stills-thu/frame789.png
53
stills-thu/frame79.png
21
EXCEPTION: No outer rectangle found
stills-thu/frame790.png
49
stills-thu/frame791.png
43
stills-thu/frame792.png
49
stills-thu/frame793.png
53
stills-thu/frame794.png
40
stills-thu/frame795.png
48
stills-thu/frame796.png
48
stills-thu/frame797.png
46
stills-thu/frame798.png
51
stills-thu/frame799.png
42
stills-thu/frame8.png
7
stills-thu/frame80.png
26
EXCEPTION: No outer rectangle found
stills-thu/frame800.png
46
stills-thu/frame801.png
49
stills-thu/frame802.png
49
stills-thu/frame803.png
48
stills-thu/frame804.png
46
stills-thu/frame805.png
52
stills-thu/frame806.png
40
stills-thu/frame807.png
51
stills-thu/frame808.png
53
stills-thu/frame809.png
44
stills-thu/frame81.png
15
stills-thu/frame810.png
44
stills-thu/frame811.png
39
stills-thu/frame812.png
50
stills-thu/frame813.png
52
stills-thu/frame814.png
45
stills-thu/frame815.png
46
stills-thu/frame816.png
60
stills-thu/frame817.png
44
stills-thu/frame818.png
46
stills-thu/fr

In [45]:
try:
    assert 1==0, "failed that"
except Exception as e:
    print(e)

failed that


In [8]:
cv2.imread("frame1.png")


array([[[196, 199, 206],
        [197, 199, 209],
        [194, 198, 208],
        ..., 
        [183, 184, 184],
        [181, 182, 182],
        [179, 180, 180]],

       [[197, 199, 209],
        [197, 199, 209],
        [197, 199, 209],
        ..., 
        [183, 184, 184],
        [181, 182, 182],
        [179, 180, 180]],

       [[197, 200, 207],
        [197, 199, 209],
        [197, 199, 209],
        ..., 
        [181, 181, 183],
        [181, 181, 183],
        [179, 179, 181]],

       ..., 
       [[ 49,  65, 112],
        [ 70,  87, 130],
        [ 77,  97, 135],
        ..., 
        [166, 175, 193],
        [164, 176, 191],
        [164, 176, 191]],

       [[ 42,  60, 109],
        [ 66,  86, 126],
        [ 90, 112, 143],
        ..., 
        [164, 175, 193],
        [164, 176, 191],
        [164, 176, 191]],

       [[ 53,  68, 111],
        [ 66,  84, 122],
        [ 83, 105, 136],
        ..., 
        [164, 175, 193],
        [164, 175, 193],
        [164, 175,